In [3]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from io import StringIO
from selenium import webdriver 

In [4]:
# years we are scraping from
years = list(range(1989, 2025))

# webscraping url
url = f"https://www.basketball-reference.com/leagues/NBA_YEAR_per_game.html"

In [6]:
# doing with selenium this time because requests doesnt work with such a big webpage
driver = webdriver.Chrome()

for year in years:
    url =  f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"
    driver.get(url)
    # just need to scroll down so the whole webpage renders before we capture it 
    driver.execute_script("window.scrollTo(1,100000)")
    time.sleep(2)

    html = driver.page_source
    with open("players/{}.html".format(year), "w+", encoding="utf-8") as file:
        file.write(html)

In [5]:
playerdfs = []
for year in years:
    with open("players/{}.html".format(year), encoding="utf-8") as file:
        page = file.read()
    soup = BeautifulSoup(page, "html.parser")
    # useless header at the top of the chart
    soup.find("tr", class_="thead").decompose()
    player_table = soup.find(id="per_game_stats")
    df = pd.read_html(StringIO(str(player_table)))[0]
    df["Year"] = year
    playerdfs.append(df)
player = pd.concat(playerdfs)
player.head()

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
0,1,Michael Jordan,25,CHI,SG,81,81,40.2,11.9,22.2,...,6.2,8.0,8.0,2.9,0.8,3.6,3.0,32.5,"MVP-2,DPOY-5,AS,NBA1",1989
1,2,Karl Malone,25,UTA,PF,80,80,39.1,10.1,19.5,...,7.4,10.7,2.7,1.8,0.9,3.6,3.6,29.1,"MVP-3,AS,NBA1",1989
2,3,Dale Ellis,28,SEA,SG,82,82,38.9,10.5,20.9,...,2.3,4.2,2.0,1.3,0.3,2.7,2.4,27.5,"AS,NBA3",1989
3,4,Clyde Drexler,26,POR,SG,78,78,39.3,10.6,21.4,...,4.2,7.9,5.8,2.7,0.7,3.2,3.4,27.2,AS,1989
4,5,Alex English,35,DEN,SF,82,82,36.5,11.3,22.9,...,2.2,4.0,4.7,0.8,0.1,2.4,2.1,26.5,AS,1989


In [6]:
player.to_csv("player.csv")